In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")#When we perform a shuffle, Spark outputs 200 shuffle partitions.
print("SparkSession created with name as 'spark'")

SparkSession created with name as 'spark'


In [13]:
df = spark.read.json("../../../../hadoopedge/bigdata/modules/spark/git/data/SparkTheDefinitiveGuide/flight-data/json/2015-summary.json")
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [22]:
df.select("DEST_COUNTRY_NAME").show()
df.filter(col("DEST_COUNTRY_NAME")=="United States").show()


+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|       United States|
|       United States|
|               Egypt|
|       United States|
|       United States|
|       United States|
|          Costa Rica|
|             Senegal|
|             Moldova|
|       United States|
|       United States|
|              Guyana|
|               Malta|
|            Anguilla|
|             Bolivia|
|       United States|
|             Algeria|
|Turks and Caicos ...|
|       United States|
+--------------------+
only showing top 20 rows

+-----------------+--------------------+-----+
|DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+-----------------+--------------------+-----+
|    United States|             Romania|   15|
|    United States|             Croatia|    1|
|    United States|             Ireland|  344|
|    United States|               India|   62|
|    United States|           Singapore|    1|
|    United States|             Grenada|   62|

In [17]:
from pyspark.sql.functions import expr, col, column ,lit
#df=spark.read.json('data/2015-summary.json')
#df=spark.read.format("json").load('data/2015-summary.json')

df.select("DEST_COUNTRY_NAME").show(2)
df.select(expr("DEST_COUNTRY_NAME")).show(2)
df.select(col("DEST_COUNTRY_NAME")).show(2)
df.select(column("DEST_COUNTRY_NAME")).show(2)

df.select(expr("DEST_COUNTRY_NAME as destination")).show(2)
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME")).show(2)
# df.select(expr = df.selectExpr (shorthand)
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)

df.selectExpr("*", "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry").show(2)
#aggregation
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)
#SELECT *, (DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry FROM dfTable LIMIT 2.

df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows

+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|   

In [18]:
df.withColumn("numberOne", lit(1)).show(2)
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME")).show(2)
df.selectExpr( "*" ,"ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME").show(2)

df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").show(2)

df.drop("ORIGIN_COUNTRY_NAME").show(2)

df.withColumn("countLong", col("count").cast("long")).drop("count").show(2)

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows

+-----------------+-------------------+-----+-----------------------------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|(ORIGIN_COUNTRY_NAME = DEST_COUNTRY_NAME)|
+-----------------+-------------------+-----+---------------------------------

#### WordCount DataFrame

In [25]:
wdf=spark.read.text("../data/NOTICE.txt")

In [31]:
from pyspark.sql.functions import split,col
wdf.select(split("value"," ")).show()

+--------------------+
| split(value,  , -1)|
+--------------------+
|[, NOTICES, AND, ...|
|[, Do, Not, Trans...|
|                  []|
|[, This, software...|
|[, open, source, ...|
|[, send, a, check...|
|[, source, compon...|
|                  []|
|[, Source, Code, ...|
|[, Microsoft, Cor...|
|[, One, Microsoft...|
|[, Redmond,, WA, ...|
|             [, USA]|
|                  []|
|[, Notwithstandin...|
|[, extent, requir...|
|[, General, Publi...|
|                  []|
|                  []|
|[----------------...|
+--------------------+
only showing top 20 rows

